In [12]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import count, when, col

In [13]:
# Build the SparkSession
spark = SparkSession.builder \
   .master("local") \
   .appName("DDAM Project") \
   .config("spark.executor.memory", "1gb") \
   .getOrCreate()
   
sc = spark.sparkContext

In [14]:
df_boxscore = spark.read.csv("data/boxscore.csv", header=True, inferSchema=True)
df_games = spark.read.csv("data/games.csv", header=True, inferSchema=True)
df_salaries = spark.read.csv("data/salaries.csv", header=True, inferSchema=True)

In [15]:
df_boxscore.show(5)

+-------+----------------+------------------+-----+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---------+
|game_id|        teamName|        playerName|   MP| FG|FGA| 3P|3PA| FT|FTA|ORB|DRB|TRB|AST|STL|BLK|TOV| PF|PTS|+/-|isStarter|
+-------+----------------+------------------+-----+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---------+
|      1|Sacramento Kings|Corliss Williamson|37:20|  7| 11|  0|  0|  0|  0|  1|  3|  4|  4|  1|  1|  4|  5| 14| -2|        1|
|      1|Sacramento Kings|    Mitch Richmond|32:00|  6| 12|  1|  4|  1|  1|  0|  5|  5|  3|  1|  2|  3|  1| 14|-12|        1|
|      1|Sacramento Kings|    Olden Polynice|31:34|  0|  4|  0|  0|  1|  4|  2|  5|  7|  3|  1|  0|  4|  3|  1|-12|        1|
|      1|Sacramento Kings|Mahmoud Abdul-Rauf|29:27|  7| 13|  1|  2|  2|  2|  0|  2|  2|  5|  1|  1|  2|  2| 17| -7|        1|
|      1|Sacramento Kings|       Brian Grant|25:13|  3| 11|  0|  0|  2|  2|  1|  5|  6|  0|  0|  2|  1|  2|  8| -7|   

In [16]:
df_games.show(5)

+---------------+--------------------+----------+--------------------+----------+----------+-----+-------+----------+---------+-------+
|seasonStartYear|            awayTeam|pointsAway|            homeTeam|pointsHome|attendance|notes|startET|  datetime|isRegular|game_id|
+---------------+--------------------+----------+--------------------+----------+----------+-----+-------+----------+---------+-------+
|           1996|    Sacramento Kings|        85|     Houston Rockets|        96|   16285.0| NULL|   NULL|1996-11-01|        1|      1|
|           1996|Los Angeles Clippers|        97|Golden State Warr...|        85|   15593.0| NULL|   NULL|1996-11-01|        1|      2|
|           1996|Portland Trail Bl...|       114| Vancouver Grizzlies|        85|   19193.0| NULL|   NULL|1996-11-01|        1|      3|
|           1996| Seattle SuperSonics|        91|           Utah Jazz|        99|   19911.0| NULL|   NULL|1996-11-01|        1|      4|
|           1996|     New York Knicks|       107

In [17]:
df_salaries.show(5)

+----------------+---------------+-----------+------------------+
|      playerName|seasonStartYear|     salary|inflationAdjSalary|
+----------------+---------------+-----------+------------------+
|  Michael Jordan|           1996|$30,140,000|       $52,258,566|
|    Horace Grant|           1996|$14,857,000|       $25,759,971|
|   Reggie Miller|           1996|$11,250,000|       $19,505,934|
|Shaquille O'Neal|           1996|$10,714,000|       $18,576,585|
|     Gary Payton|           1996|$10,212,000|       $17,706,187|
+----------------+---------------+-----------+------------------+
only showing top 5 rows



In [18]:
#removing notes and startET from df_games
columns_to_drop = ["awayTeam", "pointsAway", "homeTeam", "pointsHome", "attendance", "notes", "startET", "datetime", "isRegular"]

# Drop the specified columns
df_games = df_games.drop(*columns_to_drop)

# Show the first 5 rows of the updated df_games DataFrame
df_games.show(5)

+---------------+-------+
|seasonStartYear|game_id|
+---------------+-------+
|           1996|      1|
|           1996|      2|
|           1996|      3|
|           1996|      4|
|           1996|      5|
+---------------+-------+
only showing top 5 rows



In [19]:
#here the plot of the years (the flat line)

In [20]:

df_games = df_games.filter((col("seasonStartYear") >= 2000) & (col("seasonStartYear") <= 2010))
df_games.show(5)

+---------------+-------+
|seasonStartYear|game_id|
+---------------+-------+
|           2000|   4577|
|           2000|   4578|
|           2000|   4579|
|           2000|   4580|
|           2000|   4581|
+---------------+-------+
only showing top 5 rows



In [21]:
df_salaries = df_salaries.filter((col("seasonStartYear") >= 2000) & (col("seasonStartYear") <= 2010))
df_salaries.show(5)

+----------------+---------------+-----------+------------------+
|      playerName|seasonStartYear|     salary|inflationAdjSalary|
+----------------+---------------+-----------+------------------+
|   Kevin Garnett|           2000|$19,610,000|       $30,904,632|
|Shaquille O'Neal|           2000|$19,285,715|       $30,393,570|
| Alonzo Mourning|           2000|$16,880,000|       $26,602,253|
|    Juwan Howard|           2000|$16,875,000|       $26,594,373|
| Hakeem Olajuwon|           2000|$16,700,000|       $26,318,580|
+----------------+---------------+-----------+------------------+
only showing top 5 rows



In [22]:
#game_id that are played between 2000 and 2010
selected_game_ids = df_games.select("game_id").distinct()

#filter salaries based on game_id

df_boxscore = df_boxscore.join(selected_game_ids, "game_id")

df_boxscore.show(5)

+-------+-------------------+-------------+-----+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---------+
|game_id|           teamName|   playerName|   MP| FG|FGA| 3P|3PA| FT|FTA|ORB|DRB|TRB|AST|STL|BLK|TOV| PF|PTS|+/-|isStarter|
+-------+-------------------+-------------+-----+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---------+
|   4577|Seattle SuperSonics|  Gary Payton|48:00| 10| 26|  2|  9|  5|  7|  4| 10| 14| 10|  1|  0|  6|  4| 27| -6|        1|
|   4577|Seattle SuperSonics|    Vin Baker|37:28|  7| 16|  0|  0|  5|  6|  2|  5|  7|  1|  1|  4|  3|  3| 19|  0|        1|
|   4577|Seattle SuperSonics|Patrick Ewing|36:36|  1|  6|  0|  0|  0|  0|  2| 10| 12|  2|  0|  2|  3|  4|  2|-15|        1|
|   4577|Seattle SuperSonics|Rashard Lewis|34:32|  3| 10|  2|  5|  3|  4|  1|  4|  5|  1|  1|  0|  3|  5| 11| -4|        1|
|   4577|Seattle SuperSonics|Desmond Mason|17:01|  3|  9|  0|  3|  1|  2|  3|  2|  5|  2|  0|  1|  1|  5|  7|-15|        1|
+-------

In [23]:
#writing csv for the sharing via github
output_path = "data/df_boxscore_2000-2010"
df_boxscore.write.csv(output_path, header=True, mode="overwrite")

output_path = "data/df_salaries_2000-2010"
df_salaries.write.csv(output_path, header=True, mode="overwrite")

In [ ]:
##########################

In [38]:
from pyspark.sql.functions import col,isnan, when, count
df_boxscore.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df_boxscore.columns]
   ).show()

+-------+--------+----------+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---------+
|game_id|teamName|playerName| MP| FG|FGA| 3P|3PA| FT|FTA|ORB|DRB|TRB|AST|STL|BLK|TOV| PF|PTS|+/-|isStarter|
+-------+--------+----------+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---------+
|      0|       0|         0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0| 19|        0|
+-------+--------+----------+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---------+



In [39]:
# Create a list comprehension to count null or NaN values for each column
counts_expr = [count(when(col(c).isNull(), c)).alias(c) if c != 'datetime' else count(when(col(c).isNull(), c)).alias(c+'_null_count') for c in df_games.columns]

# Apply the counts expression to the DataFrame
df_games.select(counts_expr).show()

+---------------+--------+----------+--------+----------+----------+-----+-------+-------------------+---------+-------+
|seasonStartYear|awayTeam|pointsAway|homeTeam|pointsHome|attendance|notes|startET|datetime_null_count|isRegular|game_id|
+---------------+--------+----------+--------+----------+----------+-----+-------+-------------------+---------+-------+
|              0|       0|         0|       0|         0|         0|14245|      0|                  0|        0|      0|
+---------------+--------+----------+--------+----------+----------+-----+-------+-------------------+---------+-------+



In [40]:
from pyspark.sql.functions import col,isnan, when, count
df_salaries.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df_salaries.columns]
   ).show()

+----------+---------------+------+------------------+
|playerName|seasonStartYear|salary|inflationAdjSalary|
+----------+---------------+------+------------------+
|         0|              0|     0|                 0|
+----------+---------------+------+------------------+

